In [1]:
import sys
sys.executable

'C:\\Users\\amar\\anaconda3\\envs\\credit_risk\\python.exe'

In [2]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import os

import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE

from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score


import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("credit_risk_dataset.csv")
df1 = df.copy()
df1.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [4]:
df1["loan_grade"].value_counts()

A    10777
B    10451
C     6458
D     3626
E      964
F      241
G       64
Name: loan_grade, dtype: int64

In [5]:
df2 = pd.get_dummies(df1, columns=["person_home_ownership", "loan_intent", "cb_person_default_on_file"])

In [6]:
df2["loan_grade"].replace({"A":0, "B":1, "C":2, "D":4, "E":4, "F":4, "G":4}, inplace=True)

In [7]:
df2.head()

,person_age,person_income,person_emp_length,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,...,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,cb_person_default_on_file_N,cb_person_default_on_file_Y
0,22,59000,123.0,4,35000,16.02,1,0.59,3,0,...,0,1,0,0,0,0,1,0,0,1
1,21,9600,5.0,1,1000,11.14,0,0.10,2,0,...,1,0,0,1,0,0,0,0,1,0
2,25,9600,1.0,2,5500,12.87,1,0.57,3,1,...,0,0,0,0,0,1,0,0,1,0
3,23,65500,4.0,2,35000,15.23,1,0.53,2,0,...,0,1,0,0,0,1,0,0,1,0
4,24,54400,8.0,2,35000,14.27,1,0.55,4,0,...,0,1,0,0,0,1,0,0,0,1


In [8]:
df2["loan_status"].value_counts()

0    25473
1     7108
Name: loan_status, dtype: int64

# Impute

In [9]:
impute = KNNImputer()
array = impute.fit_transform(df2)
df3 = pd.DataFrame(array, columns=df2.columns)
df3.head()

,person_age,person_income,person_emp_length,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,...,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,cb_person_default_on_file_N,cb_person_default_on_file_Y
0,22.0,59000.0,123.0,4.0,35000.0,16.02,1.0,0.59,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,21.0,9600.0,5.0,1.0,1000.0,11.14,0.0,0.10,2.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,25.0,9600.0,1.0,2.0,5500.0,12.87,1.0,0.57,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,23.0,65500.0,4.0,2.0,35000.0,15.23,1.0,0.53,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,24.0,54400.0,8.0,2.0,35000.0,14.27,1.0,0.55,4.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [10]:
df3.isna().sum()

person_age                        0
person_income                     0
person_emp_length                 0
loan_grade                        0
loan_amnt                         0
loan_int_rate                     0
loan_status                       0
loan_percent_income               0
cb_person_cred_hist_length        0
person_home_ownership_MORTGAGE    0
person_home_ownership_OTHER       0
person_home_ownership_OWN         0
person_home_ownership_RENT        0
loan_intent_DEBTCONSOLIDATION     0
loan_intent_EDUCATION             0
loan_intent_HOMEIMPROVEMENT       0
loan_intent_MEDICAL               0
loan_intent_PERSONAL              0
loan_intent_VENTURE               0
cb_person_default_on_file_N       0
cb_person_default_on_file_Y       0
dtype: int64

In [11]:
df2["loan_status"].value_counts()

0    25473
1     7108
Name: loan_status, dtype: int64

In [12]:
x = df3.drop("loan_status", axis=1)
y = df3["loan_status"]

In [13]:
smt = SMOTE()
x_sample, y_sample = smt.fit_resample(x, y)

In [14]:
y_sample.value_counts()

1.0    25473
0.0    25473
Name: loan_status, dtype: int64

In [15]:
x = x_sample
y = y_sample

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)

knn = KNeighborsClassifier(n_neighbors=3, p=1)
knn.fit(x_train, y_train)

y_pred_train = knn.predict(x_train)
y_pred_test = knn.predict(x_test)
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print()
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

[[18485  1893]
 [ 1161 19217]]
              precision    recall  f1-score   support

         0.0       0.94      0.91      0.92     20378
         1.0       0.91      0.94      0.93     20378

    accuracy                           0.93     40756
   macro avg       0.93      0.93      0.93     40756
weighted avg       0.93      0.93      0.93     40756


[[4235  860]
 [ 611 4484]]
              precision    recall  f1-score   support

         0.0       0.87      0.83      0.85      5095
         1.0       0.84      0.88      0.86      5095

    accuracy                           0.86     10190
   macro avg       0.86      0.86      0.86     10190
weighted avg       0.86      0.86      0.86     10190



In [16]:
import pickle 
import json


In [18]:
df3.head()

,person_age,person_income,person_emp_length,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,...,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,cb_person_default_on_file_N,cb_person_default_on_file_Y
0,22.0,59000.0,123.0,4.0,35000.0,16.02,1.0,0.59,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,21.0,9600.0,5.0,1.0,1000.0,11.14,0.0,0.10,2.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,25.0,9600.0,1.0,2.0,5500.0,12.87,1.0,0.57,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,23.0,65500.0,4.0,2.0,35000.0,15.23,1.0,0.53,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,24.0,54400.0,8.0,2.0,35000.0,14.27,1.0,0.55,4.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [17]:
loan_grade = {"A":0, "B":1, "C":2, "D":4, "E":4, "F":4, "G":4}

In [19]:
list(x_sample)

['person_age',
 'person_income',
 'person_emp_length',
 'loan_grade',
 'loan_amnt',
 'loan_int_rate',
 'loan_percent_income',
 'cb_person_cred_hist_length',
 'person_home_ownership_MORTGAGE',
 'person_home_ownership_OTHER',
 'person_home_ownership_OWN',
 'person_home_ownership_RENT',
 'loan_intent_DEBTCONSOLIDATION',
 'loan_intent_EDUCATION',
 'loan_intent_HOMEIMPROVEMENT',
 'loan_intent_MEDICAL',
 'loan_intent_PERSONAL',
 'loan_intent_VENTURE',
 'cb_person_default_on_file_N',
 'cb_person_default_on_file_Y']

In [20]:
json_data = {"loan_grad":{"A":0, "B":1, "C":2, "D":4, "E":4, "F":4, "G":4},
            "columns":list(x_sample)}

In [21]:
with open("json_data.json", "w") as f:
    json.dump(json_data, f)

In [27]:
with open("model.pkl", "wb") as f:
    pickle.dump(knn, f)